In [151]:
from pathlib import Path

import pandas as pd
import pyarrow as pa
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import glob
PATH = Path('../input/optiver-realized-volatility-prediction/')
from sklearn.metrics import r2_score

In [5]:
file_path = list_order_book_file_train[0]

In [45]:
data_df = book_train.loc[book_train.stock_id == stock_ids[0]].copy()

In [58]:
def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0,600)], names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

In [75]:
book_train = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet')
stock_ids = book_train.stock_id.unique()
for stock_id in tqdm.notebook.tqdm(stock_ids):
    data_df = book_train.loc[book_train.stock_id == stock_id]
    data_df = ffill(data_df)
    table = pa.Table.from_pandas(data_df)
    pa.parquet.write_to_dataset(table, root_path='../input/optiver-realized-volatility-prediction/book_train_reindexed.parquet', partition_cols=['stock_id'])

In [77]:
reindexed_book_train =pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train_reindexed.parquet')

In [78]:
reindexed_book_train

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
2,5,2,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
3,5,3,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
4,5,4,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
...,...,...,...,...,...,...,...,...,...,...,...
257359195,32767,595,0.999527,0.999797,0.999391,0.999932,900,600,600,528,99
257359196,32767,596,0.999527,0.999797,0.999391,0.999932,900,700,600,628,99
257359197,32767,597,0.999527,0.999797,0.999391,0.999932,600,700,600,628,99
257359198,32767,598,0.999527,0.999797,0.999391,0.999932,600,700,600,628,99


In [79]:
len(reindexed_book_train) == 112 * 3830 * 600

False

In [80]:
reindexed_book_train.stock_id.nunique()

112

In [81]:
reindexed_book_train.time_id.nunique()

3830

In [87]:
counts = reindexed_book_train.groupby('stock_id')['time_id'].agg('count')

In [83]:
112 * 3830 * 600

257376000

In [88]:
counts

stock_id
0      2298000
1      2298000
10     2298000
100    2297400
101    2298000
        ...   
95     2298000
96     2298000
97     2298000
98     2298000
99     2298000
Name: time_id, Length: 112, dtype: int64

In [92]:
counts[counts != 2298000]

stock_id
100    2297400
13     2297400
38     2289000
75     2297400
80     2292000
Name: time_id, dtype: int64

In [94]:
reindexed_book_train.loc[reindexed_book_train.stock_id == 80].time_id.nunique()

3820

In [96]:
book_train.loc[book_train.stock_id == 80].time_id.nunique()

3820

In [97]:
book_train.loc[book_train.stock_id == 0].time_id.nunique()

3830

In [98]:
cnt =0
for stock_id in stock_ids:
    cnt += book_train.loc[book_train.stock_id == stock_id].time_id.nunique()
cnt * 600

257359200

In [102]:
reindexed_book_train['wap'] = (reindexed_book_train.bid_price1 * reindexed_book_train.ask_size1 + reindexed_book_train.ask_price1 * reindexed_book_train.bid_size1) / (reindexed_book_train.ask_size1 + reindexed_book_train.bid_size1)

In [128]:
def realized_volatility_single_stock(wap):
    log_wap = np.log(wap)
    log_wap = log_wap.reshape(-1,600)
    log_ret = log_wap[:,1:] - log_wap[:,:-1]
    volatility = np.sqrt((log_ret ** 2).sum(axis=1))
    return volatility

In [146]:
res = []
for stock_id in reindexed_book_train.stock_id.unique():
    data_df = reindexed_book_train[reindexed_book_train.stock_id == stock_id]
    r_v = realized_volatility_single_stock(data_df.wap.to_numpy())
    row_ids =[f'{stock_id}-{x}' for x in data_df.time_id.unique()]
    res.append(pd.DataFrame(zip(row_ids, r_v), columns = ['row_id', 'pred']))
res = pd.concat(res)

In [149]:
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_joined = train.merge(res, on = ['row_id'], how = 'left')

In [152]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')

Performance of the naive prediction: R2 score: 0.628, RMSPE: 0.341
